In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
from google.colab import drive
import pickle
import json
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Ex: sbap-core-ic50-protein-r256 (r is just powers of 2 often 2**7-10)
task = 'sbap'
noise = 'general'
measurement = 'potency'
domain = 'protein'
N = int(2**10 / .8)
r = 256
p = 0.3 # independent bernoulli prob of each cluster being included in a subset

In [ ]:
betas = np.load(f'/content/drive/MyDrive/Colab Notebooks/UNC/Comp790-142-Generative and OOD Methods/Project/data/drugood_results/{task}/{noise}/{measurement}/{domain}/diffrand/clustering-N{N}-r{r}-betas.npy')
clusters = np.load(f'/content/drive/MyDrive/Colab Notebooks/UNC/Comp790-142-Generative and OOD Methods/Project/data/drugood_results/{task}/{noise}/{measurement}/{domain}/diffrand/clustering-N{N}-r{r}-dirs.npy')

In [ ]:
print(betas.shape)
print(clusters.shape)

(1280, 2561)
(1280, 32)


In [ ]:
k = clusters.shape[1]

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(clusters, betas, test_size=0.2, random_state=42)

## Beta Model

## Train MLP on training dirs and betas

[0,0,11....1] -> beta

In [ ]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential

# Assuming 'd_beta' is the dimensionality of your betas
d_beta = betas.shape[1]  # Dimensionality of beta vectors
d = clusters.shape[1]  # Dimensionality of cluster vectors

# Define the model
mlp = Sequential([
    Dense(1024, activation='elu', input_shape=(d,)),  # First layer
    Dropout(0.5),  # Dropout for regularization
    Dense(1024, activation='elu'),  # Second layer
    Dropout(0.5),  # Dropout for regularization
    Dense(d_beta)  # Output layer matches dimensionality of betas
])

# Compile the model
mlp.compile(optimizer='adam', loss='mse')  # You might want to adjust these parameters

In [ ]:
# Fit the model on your data
mlp.fit(X_train, Y_train, epochs=10)

Epoch 1/10
32/32 [==============================] - 3s 65ms/step - loss: 0.0537
Epoch 2/10
32/32 [==============================] - 2s 63ms/step - loss: 0.0358
Epoch 3/10
32/32 [==============================] - 3s 83ms/step - loss: 0.0314
Epoch 4/10
32/32 [==============================] - 3s 86ms/step - loss: 0.0290
Epoch 5/10
32/32 [==============================] - 2s 60ms/step - loss: 0.0280
Epoch 6/10
32/32 [==============================] - 2s 62ms/step - loss: 0.0272
Epoch 7/10
32/32 [==============================] - 2s 60ms/step - loss: 0.0266
Epoch 8/10
32/32 [==============================] - 2s 66ms/step - loss: 0.0260
Epoch 9/10
32/32 [==============================] - 2s 64ms/step - loss: 0.0257
Epoch 10/10
32/32 [==============================] - 3s 89ms/step - loss: 0.0256


## Test MLP performance on held-out betas

In [ ]:
test_loss = mlp.evaluate(X_test, Y_test)
print(f"Test Loss: {test_loss}")

# To visualize predictions
predictions = mlp.predict(X_test[:5])
for i in range(predictions.shape[0]):
    print(f"Prediction {i+1}:")
    print(f"\t\tPredicted Beta: {predictions[i]}")
    print(f"\t\tActual Beta:    {Y_test[i]}")

8/8 [==============================] - 0s 11ms/step - loss: 0.0234
Test Loss: 0.02343374490737915
1/1 [==============================] - 0s 92ms/step
Prediction 1:
		Predicted Beta: [ 0.04201026  0.12212431 -0.16508976 ...  0.10026436 -0.40697125
 -0.01320569]
		Actual Beta:    [ 0.142752    0.08215274 -0.24733557 ...  0.01048001 -0.33449138
 -0.00481456]
Prediction 2:
		Predicted Beta: [ 0.02989113  0.1495587  -0.19950432 ... -0.05615363 -0.27765688
 -0.00205966]
		Actual Beta:    [ 0.02032017  0.1481269  -0.05195085 ...  0.00312812 -0.16535027
  0.01484797]
Prediction 3:
		Predicted Beta: [-0.0185276   0.02067932  0.08590043 ... -0.00699385 -0.180829
 -0.0126144 ]
		Actual Beta:    [ 0.10172602 -0.1706423   0.3861933  ... -0.3812952  -0.14867297
 -0.0202467 ]
Prediction 4:
		Predicted Beta: [ 0.04323824  0.04415898  0.01589024 ...  0.11466277 -0.1836502
 -0.0037879 ]
		Actual Beta:    [ 0.00707817  0.05152297  0.01279808 ...  0.00068223 -0.1902849
  0.01877642]
Prediction 5:
		Predic

## Generate New Betas

In [ ]:
# Function to generate new betas from the MLP model
def generate_new_betas(model, random_dirs):
    return model.predict(random_dirs)

# Generating new betas
M = 100_000  # Number of betas to generate

# Generate random k-dimensional binary vectors
p = 0.3 # Independent Bernoulli Probability of each cluster
gen_cluster_vectors = np.random.binomial(1, p, size=(M, k))  # M random subsets
drawn_betas = generate_new_betas(mlp, gen_cluster_vectors)

3125/3125 [==============================] - 37s 12ms/step


In [ ]:
print(drawn_betas.shape)

(100000, 2561)


In [ ]:
print(np.mean(betas))
print(np.mean(drawn_betas))

0.0005995804935047487
0.0002893208
